1. Import necessart visualisation library and define inputs simulation and additional vector to add

In [1]:
from hat.visualisation import NotebookMap

stations= '~/destinE/outlets_v4.0_20230726_withEFAS.csv'
observations= '~/destinE/destine_observation.nc'

simulations = {
    "i05j": "~/destinE/cama_i05j_stations.nc",
    "i05h": "~/destinE/cama_i05h_stations.nc"
}

config = {
    "station_epsg": 4326,
    "station_id_column_name": "station_id",
    "station_filters":"station_id_num <= 5000",
    "station_coordinates": ["StationLon", "StationLat"],
    "obs_var_name": "obsdis",
    "sim_var_name": "dis"
}

statistics = {
    "i05j": "~/destinE/statistics_i05j.nc",
    "i05h": "~/destinE/statistics_i05h.nc"
}

2. Initialise & processing all the input data into the map object

In [2]:
map = NotebookMap(config, stations, observations, simulations, stats=statistics)

Found 49 common stations


3. Display map interface for interactive viewing

In [3]:
map.mapplot(colorby='kge', sim='i05h', range=[-1, 1])

Initialising plotly


In [4]:
map.bounds[0][0]


38.13340834